# Train OCR text Detector quick example

For train datasets please download last version of ocr datasets [https://nomeroff.net.ua/datasets/](https://nomeroff.net.ua/datasets/). Unpack archive and rename to **./datasets/ocr** .
For examle
```bash
cd ./datasets/ocr
wget https://nomeroff.net.ua/datasets/autoriaNumberplateOcrUa-2019-02-19.zip
unzip autoriaNumberplateOcrUa-2019-02-19.zip
mv autoriaNumberplateOcrUa-2019-02-19 ua
```
or use your own dataset.

In [10]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

import keras
keras.backend.clear_session()

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../')

DATASET_NAME = "ru"
VERSION = "3"
MODE = "cpu"
PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "datasets/ocr/", DATASET_NAME)
RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/", 'anpr_ocr_{}_{}-{}.h5'.format(DATASET_NAME, VERSION, MODE))

FROZEN_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/", 'anpr_ocr_{}_{}-{}.pb'.format(DATASET_NAME, VERSION, MODE))

sys.path.append(NOMEROFF_NET_DIR)

from NomeroffNet.Base import OCR, convert_keras_to_freeze_pb

In [11]:
class ru(OCR):
    def __init__(self):
        OCR.__init__(self)
        # only for usage model
        # in train generate automaticly
        self.letters = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "E", "H", "K", "M", "O", "P", "T", "X", "Y"]
        
        self.EPOCHS = 1

In [12]:
ocrTextDetector = ru()
model = ocrTextDetector.prepare(PATH_TO_DATASET, aug_count=0)

GET ALPHABET
Max plate length in "val": 9
Max plate length in "train": 9
Max plate length in "test": 9
Max plate length in train, test and val do match
Letters in train, val and test do match
Letters: 0 1 2 3 4 5 6 7 8 9 A B C E H K M O P T X Y

EXPLAIN DATA TRANSFORMATIONS
Text generator output (data which will be fed into the neutral network):
1) the_input (image)
2) the_labels (plate number): M004BY199 is encoded as [16, 0, 0, 4, 11, 21, 1, 9, 9]
3) input_length (width of image that is fed to the loss function): 30 == 128 / 4 - 2
4) label_length (length of plate number): 9
START BUILD DATA
DATA PREPARED


In [14]:
#model = ocrTextDetector.load(RESULT_MODEL_PATH)
#RESULT_MODEL_PATH

In [8]:
model = ocrTextDetector.train(mode=MODE, is_random=1)


START TRAINING
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input[0][0]                  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
_____________________________________________________________________________________________

In [15]:
ocrTextDetector.test(verbose=True)


RUN TEST

Predicted: 		 O653BX125
True: 			 H047BY125

Predicted: 		 B639MY16
True: 			 B639MY116

Predicted: 		 O536OO48
True: 			 O536OO43

Predicted: 		 M283YH31
True: 			 P709AH31

Predicted: 		 B936TM86
True: 			 B956TA86

Predicted: 		 Y512PK52
True: 			 Y542PK52

Predicted: 		 T659AP197
True: 			 T689AP197

Predicted: 		 H120HH32
True: 			 H120HH39

Predicted: 		 Y013YY6
True: 			 Y013YY26

Predicted: 		 X303PX02
True: 			 X303PK02

Predicted: 		 A302YT15
True: 			 A302TT152

Predicted: 		 X518XX51
True: 			 X518XX61

Predicted: 		 X123HK62
True: 			 Y123HK62

Predicted: 		 H577AMP197
True: 			 H577AP197

Predicted: 		 X193BH19
True: 			 X193BH197

Predicted: 		 E5B2PO46
True: 			 E282PO46

Predicted: 		 PM813KK31
True: 			 M813KK31

Predicted: 		 K707KH8
True: 			 K707KP58
acc: 0.9633401221995926


In [10]:
ocrTextDetector.save(RESULT_MODEL_PATH, verbose=True)

SAVED TO /mnt/data/var/www/html2/js/nomeroff-net/models/anpr_ocr_ru_3-cpu.h5


In [13]:
RESULT_MODEL_PATH

'/mnt/data/var/www/html2/js/nomeroff-net/models/anpr_ocr_ru_3-cpu.h5'

In [14]:
ocrTextDetector.load(RESULT_MODEL_PATH)

### Convert keras OCR  .h5 model to .pb graph

In [16]:
import keras
keras.backend.clear_session()
model = ocrTextDetector.load(RESULT_MODEL_PATH)
convert_keras_to_freeze_pb(model, FROZEN_MODEL_PATH)

OUTPUT: softmax_ru/truediv
INPUT: the_input_ru
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input_ru (InputLayer)       (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input_ru[0][0]               
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
______________________________________________________________

In [3]:
# import keras
# PATH = "../models/anpr_ocr_ru_3-cpu.h5"

# model = keras.models.load_model(PATH, compile=False)
# model.summary()

# inp = model.get_layer("the_input")
# out = model.get_layer("softmax")

# # set new names
# inp.name = "the_input_ru"
# out.name = "softmax_ru"

# net_inp = inp.input
# net_out = out.output
# model = keras.Model(input=net_inp, output=net_out)

# model.summary()

# model.save(PATH)
